# Logistic Regression
Logistic regression with different data sets

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from utils import *
import copy
import math

%matplotlib inline

Loading the dataset 

In [6]:
X_train, y_train = load_data('ex2data1.txt')


In [14]:
#first five elements is 
print(f"First five elements are : \n {X_train[:5]}")
print(f"Type of X_train : {type(X_train)}")
print(f"Shape of X_train is : {X_train.shape}")

First five elements are : 
 [[34.62365962 78.02469282]
 [30.28671077 43.89499752]
 [35.84740877 72.90219803]
 [60.18259939 86.3085521 ]
 [79.03273605 75.34437644]]
Type of X_train : <class 'numpy.ndarray'>
Shape of X_train is : (100, 2)
